## Inference Notebook

In [8]:
# %% Cell: Inference on a new WAV file
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, models

In [14]:
# 1) Load YAMNet from TF‑Hub
yamnet = hub.load('https://tfhub.dev/google/yamnet/1')

# 2) Wrap your SavedModel for inference
#    (replace 'serving_default' if your signature is named differently)
scream_layer = tf.keras.layers.TFSMLayer(
    'human_scream_detector',
    call_endpoint='serving_default'
)

In [15]:
# 3) Audio loader
def load_wav_16k_mono(filename: str) -> np.ndarray:
    wav, _ = librosa.load(filename, sr=16000, mono=True)
    return wav.astype('float32')

In [16]:
# 4) Inference fn
def predict_scream(file_path: str, threshold: float = 0.5) -> dict:
    wav = load_wav_16k_mono(file_path)
    waveform = tf.convert_to_tensor(wav, dtype=tf.float32)
    _, embeddings, _ = yamnet(waveform)
    clip_emb = tf.reduce_mean(embeddings, axis=0, keepdims=True)  # (1,1024)

    # get probability from your SavedModel
    prob = float(scream_layer(clip_emb).numpy()[0,0])
    label = 'Scream' if prob >= threshold else 'Non‑Scream'
    return {'file': file_path, 'probability': prob, 'prediction': label}

In [18]:
# single‐file inference
result = predict_scream('audio.wav')
print(f"{result['file']}: {result['prediction']} (p={result['probability']:.3f})")

AttributeError: 'dict' object has no attribute 'numpy'